# 🚚 Logística y Transporte: costo por entrega, SLA, ocupación
KPIs: costo/km, OTIF, ocupación, distancia vacía, tiempo de ciclo.

## Casos de uso
- Ruteo óptimo con ventanas de tiempo
- Predicción de ETA y retrasos
- Optimización de ocupación y backhauling
- Control de temperatura y seguridad de carga

In [ ]:
# Estimación simple de ETA
def eta(dist_km, vel_kmh=50):
    return round(dist_km / max(vel_kmh,1), 2)
eta(120)

## Arquitectura
- Telemetría IoT y mapas, enriquecimiento geoespacial
- Optimización y simulación de rutas
- APIs para clientes y tableros de flota

## 💼 Valor en Logística
- OTIF ↑ y costo por entrega ↓ con ruteo óptimo.
- Menos tiempos muertos y backhauling inteligente.
- Trazabilidad y seguridad de carga.


In [ ]:
# Práctica: ETA y OTIF (toy)
import pandas as pd
entregas = pd.DataFrame({
    'dist_km':[10, 25, 60, 5],
    'ventana_horas':[0.5, 1.0, 1.5, 0.3],
    'vel_kmh':[40, 50, 45, 30],
})
entregas['ETA_h'] = entregas['dist_km']/entregas['vel_kmh']
entregas['OTIF'] = (entregas['ETA_h'] <= entregas['ventana_horas']).astype(int)
print(entregas)
print('OTIF %:', round(100*entregas['OTIF'].mean(),1))


## 🏗️ Arquitectura de telemetría y ruteo
- Telemetría IoT + mapas; enriquecimiento geoespacial.
- Optimizadores de rutas y simulación; tableros de flota.
- APIs de seguimiento y notificaciones a cliente.


In [ ]:
# Visual: distribución de ETA y OTIF
import plotly.express as px
fig = px.histogram(entregas, x='ETA_h', color=entregas['OTIF'].map({1:'OTIF',0:'No OTIF'}), title='ETA y cumplimiento OTIF')
fig.show()


## ✅ Checklist de estrategia de negocio y datos
- Objetivo (OTIF↑, costo/entrega↓, ocupación↑): …
- KPIs: OTIF, costo/km, ETA, ocupación, distancia vacía.
- Palancas: ruteo, telemetría, simulación, backhauling.
- Datos: GPS/IoT, órdenes, clima; calidad y latencia.
- Gobierno: dominios, seguridad carga.
- SLOs: latencia ETA, costo, disponibilidad.
- ROI/supuestos: …
- Riesgos: tráfico, clima, seguridad.


## 🔗 Puente Estrategia ↔ Ingeniería de Datos
- OKR: OTIF ≥ 95% y costo/entrega -10% → KPI negocio: OTIF, costo/km → KPI datos: precisión ETA, cobertura telemetría → Capacidades: IoT + ruteo + simulación → Decisión: ruteo y asignación de flota → Impacto: costo↓ y NPS↑.

```
[Telemetría]→[Geo-enriq.]→[ETA/Capacidad]→[OTIF]→[Alertas]→[Ruteo óptimo]→[Δ costo]
```


In [ ]:
# contrato_de_datos (logística)
contrato_datos = {
    "objetivo_negocio": "OTIF ≥ 95%",
    "kpi_negocio": "OTIF %",
    "kpi_datos": {"prec_eta_mae_min": "<5", "cobertura_gps_%": ">=98%"},
    "dataset": ["telemetria", "ordenes_envio", "geocoding"],
    "propietario": "Dominio Operaciones Logística",
    "slo": {"latencia_tracking_s": "<=60"},
    "decision": "Ruteo y asignación de flota",
    "impacto_economico_usd": 160000,
}
print(contrato_datos)


## 📋 Caso de uso: Optimización de ruteo con telemetría y ML

**Contexto de negocio:** Una empresa de logística tiene OTIF de 88% y costo promedio de $45/entrega. Objetivo: OTIF ≥95% y costo <$40.

**Problema sin ingeniería de datos:**
- Rutas planificadas manualmente en Excel.
- GPS sin integración con sistema de despacho.
- ETA calculado con velocidad promedio fija (sin considerar tráfico real).

**Solución con ingeniería de datos:**
1. Ingesta de telemetría: GPS en tiempo real + enriquecimiento con APIs de tráfico/clima.
2. Pipeline de geo-procesamiento: cálculo de distancias reales, ventanas de tiempo, capacidad de vehículos.
3. Modelo de optimización: algoritmo de ruteo con restricciones (VRP) ejecutado cada 15 min.
4. Predicción de ETA: ML entrenado con históricos de tiempos reales por ruta/hora/día.
5. API de tracking para clientes: visibilidad en tiempo real de su pedido.

**Resultado:**
- OTIF: 88% → 96.5%.
- Costo por entrega: $45 → $38 (mejor ocupación, menos km vacíos).
- ETA accuracy: 65% → 91% (predicciones a ±10 min).
- NPS de clientes: +18 puntos por visibilidad y puntualidad.
- Ahorro anual: $3.2M en costos operativos.


In [ ]:
# Ejercicio: cálculo de cumplimiento OTIF y costo por entrega
import pandas as pd

# Simular entregas con telemetría
entregas = pd.DataFrame({
    'entrega_id': range(1, 11),
    'dist_km': [15, 8, 25, 30, 12, 40, 18, 22, 10, 35],
    'tiempo_real_min': [25, 12, 38, 55, 18, 70, 28, 35, 15, 60],
    'ventana_promesa_min': [30, 15, 40, 50, 20, 65, 30, 40, 18, 55],
    'costo_combustible': [8, 4, 12, 15, 6, 20, 9, 11, 5, 18],
    'costo_mano_obra': [12, 12, 12, 18, 12, 18, 12, 12, 12, 18],
})

# KPIs de negocio
entregas['OTIF'] = (entregas['tiempo_real_min'] <= entregas['ventana_promesa_min']).astype(int)
entregas['costo_total'] = entregas['costo_combustible'] + entregas['costo_mano_obra']

otif_pct = entregas['OTIF'].mean() * 100
costo_promedio = entregas['costo_total'].mean()

print(f"OTIF: {otif_pct:.1f}%")
print(f"Costo promedio por entrega: ${costo_promedio:.2f}")
print(f"\nEntregas fuera de ventana: {(entregas['OTIF'] == 0).sum()}")
print("\n💡 Pipeline de optimización puede mejorar estos KPIs en 5-8% con datos históricos")


---

## 🧭 Navegación

**← Anterior:** [🏭 Industria y Manufactura: OEE, calidad y throughput](07_industria_manufactura.ipynb)

**Siguiente →:** [🌾 Agro y Alimentos: rendimiento, calidad y trazabilidad →](09_agro_alimentos.ipynb)

**📚 Índice de Negocio LATAM:**
- [🎯 Estrategia de Datos en LATAM: de la visión al ROI](01_estrategia_datos_latam.ipynb)
- [🛒 Retail y Consumo Masivo: demanda, surtido y pérdidas](02_retail_consumo_masivo.ipynb)
- [💳 Finanzas y Banca: riesgo, fraude, rentabilidad](03_finanzas_banca.ipynb)
- [🏥 Salud y Farmacéutico: calidad, trazabilidad, acceso](04_salud_farmaceutico.ipynb)
- [⚡ Energía y Recursos Naturales: continuidad, costos y seguridad](05_energia_recursos_naturales.ipynb)
- [📶 Telecomunicaciones: churn, ARPU y calidad de red](06_telecomunicaciones.ipynb)
- [🏭 Industria y Manufactura: OEE, calidad y throughput](07_industria_manufactura.ipynb)
- [🚚 Logística y Transporte: costo por entrega, SLA, ocupación](08_logistica_transporte.ipynb) ← 🔵 Estás aquí
- [🌾 Agro y Alimentos: rendimiento, calidad y trazabilidad](09_agro_alimentos.ipynb)
- [🏛️ Sector Público y Gobierno: transparencia, servicio y eficiencia](10_sector_publico_gobierno.ipynb)

**🎓 Otros Niveles:**
- [Nivel Junior](../nivel_junior/README.md)
- [Nivel Mid](../nivel_mid/README.md)
- [Nivel Senior](../nivel_senior/README.md)
- [Nivel GenAI](../nivel_genai/README.md)
- [Negocio LATAM](../negocios_latam/README.md)
